## Prepared the using data

In [2]:
import json
import nltk
#save the content data into format like following
#[{'twitter_content': 'text,text,text,text', 'hashtag_content': '#whatthewaht'},...]
content_data =[]
with open('smallTwitter.json') as data_file:
    for line in data_file:
        content_dic = {}
        if line[0] == '{':
            line = line[:-1]
            if line[-1] == ',':
                #get the content
                content_dic['twitter_content'] =json.loads(line[:-1]).get('json').get('text')
                if len(json.loads(line[:-1]).get('json').get('entities').get('hashtags')) !=0:
                    content_dic['hashtag_content'] = json.loads(line[:-1]).get('json').get('entities').get('hashtags')[0].get('text')
                content_data.append(content_dic)

#create the own word set from the twitter content to expand NLTK wordset
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()

words_set = []
for twitter in content_data:
    words_set +=(word_tokenizer.tokenize(twitter['twitter_content']))
words_set = list(set(words_set))

from string import punctuation  
from nltk.corpus import stopwords 

stop_words = stopwords.words('english')
non_words = list(punctuation)
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

#only need the alphabetic word
formartted_twitter_words_set = []
for word in words_set:
    if (word.isalpha() != False) and (word not in non_words) and (word not in stop_words):
        formartted_twitter_words_set.append(lemmatizer.lemmatize(word))


IOError: [Errno 2] No such file or directory: 'smallTwitter.json'

In [2]:
import nltk
nltk_words_set = list(set(nltk.corpus.words.words()))
#training whole set
training_set = formartted_twitter_words_set+nltk_words_set

#split the word in hashtags by using the training set
#using backward MaxMatch
def find_match_word(hash_content, wordlist):
    split_words = []
    while len(hash_content) !=0:
        #return the index of the matched word
        word, index = check_match(hash_content,wordlist)
        split_words.append(word)
        #remove the matched words from the original tokens
        hash_content = hash_content[len(hash_content)*(-1):index]
    return split_words

#use WordNetLemmatizer to lemmatize the word
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def check_match(hash_content,wordlist):
    count = 0
    i = 0
    j = len(hash_content)*(-1)
    temp = ''
    while i>=j:
        i -= 1
        #save the temporary word and wait for max match
        if lemmatizer.lemmatize(hash_content[i:]) in wordlist and i!=j:
            temp = hash_content[i:]
            count = i
        #if already rich the max lenght, return current saved temporary word
        elif lemmatizer.lemmatize(hash_content[i:]) in wordlist and i==j:
            temp = hash_content[i:]
            return temp,i
        else:
            #base case: if reach the maximum length and the word is not in wordlist
            #return current temporary word
            if len(temp)>0 and i==j:
                return temp,count
            #return one single letter if theres no match
            elif len(temp) == 0 and i == j:
                return hash_content[i:],-1
            else:
                continue


## Split the hash tag content 
It takes a bit long. May need map reduce or HPC to improve

In [36]:
#split the hashtag content now!!
for twitter in content_data:
    if 'hashtag_content' in twitter.keys():
        twitter['hashtag_content'] = find_match_word(twitter['hashtag_content'].lower(),training_set)


## LDA topic detection

In [37]:
import gensim
from string import punctuation  
from nltk.corpus import stopwords 

stop_words = stopwords.words('english')
non_words = list(punctuation)
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()


def process_content(tweeter_content):
    #only need the alphabetic word
    formartted_twitter_content = []
    tweeter_content =word_tokenizer.tokenize(tweeter_content)
    for word in tweeter_content:
        if (word.isalpha() != False) and (word not in non_words) and (word not in stop_words):
            formartted_twitter_content.append(lemmatizer.lemmatize(word))
    return formartted_twitter_content

In [38]:
texts = []
for twitter in content_data:
    if 'hashtag_content' in twitter.keys():
        temp_content = process_content(twitter['twitter_content']+' '.join(twitter['hashtag_content']))
        #temp_content = ' '.join(temp_content)
        texts.append(temp_content)
    else:
        temp_content =process_content(twitter['twitter_content'])
        #temp_content = ' '.join(temp_content)
        texts.append(temp_content)

In [42]:
print len(texts)

9998


In [43]:
from gensim import corpora, models
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=998, id2word = dictionary, passes=20)

/Users/DerekWang/anaconda/lib/python2.7/site-packages/gensim/models/ldamodel.py:694: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [46]:
print ldamodel.print_topics()

[(383, u'0.217*"selfie" + 0.147*"co" + 0.147*"http" + 0.115*"wedding" + 0.042*"Baby" + 0.027*"Novotel" + 0.017*"recipient" + 0.006*"Norwest" + 0.006*"samptomorrowfund" + 0.006*"tomorrowmaker"'), (581, u'0.107*"project" + 0.075*"http" + 0.074*"co" + 0.059*"shooting" + 0.058*"sm" + 0.056*"September" + 0.029*"om" + 0.027*"att" + 0.017*"eller" + 0.008*"MishBridges"'), (184, u'0.411*"bad" + 0.237*"Lol" + 0.077*"LA" + 0.000*"I" + 0.000*"go" + 0.000*"want" + 0.000*"isdonn" + 0.000*"mackaysuzie" + 0.000*"Alan" + 0.000*"notgavin"'), (207, u'0.252*"medium" + 0.074*"coverage" + 0.057*"cost" + 0.049*"traffic" + 0.048*"profile" + 0.045*"dollar" + 0.037*"Birmingham" + 0.035*"location" + 0.026*"Saturdays" + 0.019*"ffs"'), (447, u'0.147*"final" + 0.094*"eat" + 0.089*"http" + 0.089*"co" + 0.084*"Asian" + 0.079*"Appliance" + 0.061*"journey" + 0.028*"lime" + 0.018*"chilli" + 0.015*"obsessed"'), (795, u'0.059*"co" + 0.059*"http" + 0.056*"chingchongchinagirl" + 0.056*"vivilee" + 0.056*"sensationalviv" + 0.

## Sentiment Analysis

In [3]:
from string import punctuation  
from nltk.corpus import stopwords 

stop_words = stopwords.words('english')
non_words = list(punctuation)
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()


def process_content(tweeter_content):
    #only need the alphabetic word
    formartted_twitter_content = []
    tweeter_content =word_tokenizer.tokenize(tweeter_content)
    for word in tweeter_content:
        if (word.isalpha() != False) and (word not in non_words) and (word not in stop_words):
            formartted_twitter_content.append(lemmatizer.lemmatize(word))
    return formartted_twitter_content

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import OrderedDict
sid = SentimentIntensityAnalyzer()

def sort_orderedDict(orderdict):
    return OrderedDict(sorted(orderdict.items(), key = lambda x:x[1], reverse = True))

positive_tweets =[]
negative_tweets =[]
neutral_tweets = []
compound_tweets = []


for twitter in content_data:
    if 'hashtag_content' in twitter.keys():
        temp_content = process_content(twitter['twitter_content']+' '.join(twitter['hashtag_content']))
        temp_content = ' '.join(temp_content)
        result = sort_orderedDict(sid.polarity_scores(temp_content))
        if result.keys()[0] == 'pos':
            positive_tweets.append(temp_content)
        elif result.keys()[0] == 'neg':
            negative_tweets.append(temp_content)
        elif result.keys()[0] == 'neu':
            neutral_tweets.append(temp_content)
        elif result.keys()[0] == 'compound':
            compound_tweets.append(temp_content)
    else:
        temp_content =process_content(twitter['twitter_content'])
        temp_content = ' '.join(temp_content)
        result = sort_orderedDict(sid.polarity_scores(temp_content))
        if result.keys()[0] == 'pos':
            positive_tweets.append(temp_content)
        elif result.keys()[0] == 'neg':
            negative_tweets.append(temp_content)
        elif result.keys()[0] == 'neu':
            neutral_tweets.append(temp_content)
        elif result.keys()[0] == 'compound':
            compound_tweets.append(temp_content)

/Users/DerekWang/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
print len(positive_tweets)
print len(negative_tweets)
print len(neutral_tweets)
print len(compound_tweets)

438
368
7877
1315


In [32]:
print neutral_tweets[300]

Good morning Brisbane Sunrise noon sunset AEST UTC March Day length sunrise


In [35]:
print lemmas_of_a_word('UTC')

[Lemma('coordinated_universal_time.n.01.coordinated_universal_time'), Lemma('coordinated_universal_time.n.01.UTC')]


## Subject oriented program

In [4]:
from nltk.corpus import wordnet as wn
from collections import OrderedDict

#check the lemmas for a given word
def lemmas_of_a_word(word):
    lemmas = []
    for item in wn.synsets(word):
        for lemma in item.lemmas():
            lemmas.append(lemma)
    return lemmas

#check the matched lemmas for given lemmas
def matched_lemma(lemmas, word):
    match_lemma =[]
    for lemma in lemmas:
        if lemma.name().lower() == word.lower() or lemma.name() == word:
            match_lemma.append(lemma)
    return match_lemma

#filter out the primary senses
def primary_sense(word):
    lemmas_order_dic ={}
    for item in matched_lemma(lemmas_of_a_word(word),word):
        lemmas_order_dic[item.key()] = item.count()
    return lemmas_order_dic

#sort the dict to filter the primary sense
def sort_orderedDict(orderdict):
    return OrderedDict(sorted(orderdict.items(), key = lambda x:x[1], reverse = True))


#store the sysnset for the most common sense
def store_synset_primarySense(word):
    result = {}
    check_item = sort_orderedDict(primary_sense(word.lower()))
    if len(check_item)==1:
        if wn.lemma_from_key(check_item.keys()[0]).synset().pos() == 'n' or wn.lemma_from_key(check_item.keys()[0]).synset().pos() == 'v':
                result[word] = wn.lemma_from_key(check_item.keys()[0]).synset()
    elif len(check_item)>1:
        for index in range(len(check_item.keys())):
            if wn.lemma_from_key(check_item.keys()[index]).synset().pos() == 'n' or wn.lemma_from_key(check_item.keys()[index]).synset().pos() == 'v':
                result[word] = wn.lemma_from_key(check_item.keys()[index]).synset()
                continue
            else:
                pass
    return result



from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

#use the lemmatizer defined in the previous workshop
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

In [5]:
print lemmas_of_a_word("Trump")

[Lemma('trump.n.01.trump'), Lemma('trump.n.01.trump_card'), Lemma('trump.n.02.trump'), Lemma('cornet.n.01.cornet'), Lemma('cornet.n.01.horn'), Lemma('cornet.n.01.trumpet'), Lemma('cornet.n.01.trump'), Lemma('trump.v.01.trump'), Lemma('outdo.v.02.outdo'), Lemma('outdo.v.02.outflank'), Lemma('outdo.v.02.trump'), Lemma('outdo.v.02.best'), Lemma('outdo.v.02.scoop'), Lemma('trump.v.03.trump'), Lemma('trump.v.03.ruff'), Lemma('trump.v.04.trump'), Lemma('trump.v.04.trump_out')]


Test on a simple case

In [20]:
test_data1 =  ['killer']
test_data2 =  ['murder']
test_data3 =  ['bomb']
test_data4 =  ['accident']
test_data5 =  ['theft']
test_data6 =  ['robbery']
test_data7 =  ['accident','happen','in','melbourne']

def cal_happiness(test_data):
    twitter_synset =[]
    for word in test_data:
    #print word
        synset = store_synset_primarySense(lemmatize(word))
        if (len(synset) >0):
            twitter_synset.append(synset)
    
    wu_palmer_score = 0
    wu_palmer_score1 =0
    wu_palmer_score2 =0
    wu_palmer_score3 =0
    wu_palmer_score4 =0
    wu_palmer_score5 = 0
    wu_palmer_score6 =0
    for word in twitter_synset:
        wu_palmer_score += wn.wup_similarity(word.values()[0],store_synset_primarySense('crime').values()[0])
        wu_palmer_score1 += wn.wup_similarity(word.values()[0], store_synset_primarySense('homicide').values()[0])
        wu_palmer_score2 += wn.wup_similarity(word.values()[0], store_synset_primarySense('robbery').values()[0])
        wu_palmer_score3 += wn.wup_similarity(word.values()[0], store_synset_primarySense('weapon').values()[0])
        wu_palmer_score4 += wn.wup_similarity(word.values()[0], store_synset_primarySense('burglary').values()[0])
        wu_palmer_score5 += wn.wup_similarity(word.values()[0], store_synset_primarySense('danger').values()[0])
        wu_palmer_score6 += wn.wup_similarity(word.values()[0], store_synset_primarySense('abduction').values()[0])
    total = wu_palmer_score+wu_palmer_score1+wu_palmer_score2+wu_palmer_score3+wu_palmer_score4+wu_palmer_score5+wu_palmer_score6
    return float(total/7)/float(len(twitter_synset))

print 'similarity between crime and killer: ' + str(cal_happiness(test_data1))
print 'similarity between crime and murder: ' + str(cal_happiness(test_data2))
print 'similarity between crime and bomb: ' + str(cal_happiness(test_data3))
print 'similarity between crime and accident: ' + str(cal_happiness(test_data4))
print 'similarity between crime and theft: ' + str(cal_happiness(test_data5))
print 'similarity between crime and robbery: ' + str(cal_happiness(test_data6))
print 'similarity between crime and sentence(accident happen in melbourne): ' + str(cal_happiness(test_data7))

similarity between crime and killer: 0.325044754744
similarity between crime and murder: 0.121953693758
similarity between crime and bomb: 0.402373581011
similarity between crime and accident: 0.453937472425
similarity between crime and theft: 0.595804068802
similarity between crime and robbery: 0.549283046552
similarity between crime and sentence(accident happen in melbourne): 0.256422182924


In [8]:
print cal_happiness([
    "Designers",
    "create",
    "Ikea",
    "instruction",
    "billion",
    "border",
    "wall",
    "http",
    "co",
    "jLWthnaQdV"
  ])

0.191877560741


In [9]:
print store_synset_primarySense('homicide')

{'homicide': Synset('homicide.n.01')}
